In [1]:
import pandas as pd

In [2]:
from json import JSONDecoder
j = JSONDecoder()
girls = j.decode(open('./filtered_users/test_20.02.19-13:49_girl.json','r').read())
pops = j.decode(open('./filtered_users/test_20.02.19-13:49_pop.json','r').read())
print(f"girls: {len(girls)}, pops: {len(pops)}")

girls: 15, pops: 99


In [3]:
girls

[259.0,
 1219.0,
 3479.0,
 5385.0,
 6154.0,
 7916.0,
 8651.0,
 8707.0,
 9139.0,
 9143.0,
 9451.0,
 9454.0,
 10210.0,
 10212.0,
 11012.0]

In [4]:
source_comments = pd.read_csv('./wall/girl_wall/comments/test_20.02.19-13:49_girl.csv', index_col=0)
source_answers = pd.read_csv('./wall/girl_wall/answers/test_20.02.19-13:49_girl.csv', index_col=0)
source_posts = pd.read_csv('./wall/girl_wall/posts/test_20.02.19-13:49_girl.csv', index_col=0)
source_comments.replace({pd.np.NaN: None}, inplace=True)
source_answers.replace({pd.np.NaN: None}, inplace=True)
source_posts.replace({pd.np.NaN: None}, inplace=True)
source_comments.head()

,comment_id,date,from_id,num_answer,owner_id,parents_stack,post_id,reply_to_comment,reply_to_user,text
0,4212,1549722387,520143737,0,5385,[],4211,None,None,Зачем туда идти 😊
1,4206,1547927323,4014996,1,5385,[],4205,None,None,"Опасные русские медведи, о которых ходят леген..."
2,4214,1550430898,200531929,0,5385,[],4202,None,None,Мне такое чудо нужен!
3,4215,1550473100,5385,0,5385,[],4202,None,None,Котик уже нашел дом👌👌👌
4,4203,1547846810,525460657,0,5385,[],4192,None,None,Круто)


In [26]:
source_comments.sort_values(by=['post_id', 'date'] , inplace=True)
source_answers.sort_values(by=['post_id', 'date'] , inplace=True)
source_answers.head()

,comment_id,date,from_id,num_answer,owner_id,parents_stack,post_id,reply_to_comment,reply_to_user,text
2,4169,1550421347,6154,0,6154,[4167],3466,None,None,"[id51816062|Марина], сейчас да, раньше просто ..."
34,3876,1544340697,10210,0,10210,[3875],3731,None,None,Стараюсь)
31,3910,1547728110,10210,0,10210,[3909],3848,None,None,"[id3171305|Роман], ато:) приедешь посмотришь ))"
32,3911,1547729010,3171305,0,10210,[3909],3848,None,None,Потискаю))))
33,3912,1547729245,10210,0,10210,[3909],3848,3911,3.17130e+06,"[id3171305|Роман], договорились:)"


In [20]:
#base
already_used_id = []
dialogs = []
comments = source_comments.copy()

comments = comments[~comments['comment_id'].isin(already_used_id)]
anchor = comments[list(map(bool, comments['reply_to_comment']))].head(1)
while(not anchor['reply_to_comment'].dropna().empty):
    dialog = []
    comments = comments[~comments['comment_id'].isin(already_used_id)]
    anchor = comments[list(map(bool, comments['reply_to_comment']))].head(1)
    already_used_id.append(int(anchor['comment_id']))
    first = comments[comments['comment_id'] == int(anchor['reply_to_comment'])].head(1)
    dialog.append(first)
    dialog.append(anchor)
    after_anchor = comments[comments['reply_to_comment'] == int(anchor['comment_id'])].head(1)
    while(not after_anchor.dropna().empty):
        anchor = after_anchor
        already_used_id.append(int(anchor['comment_id']))
        dialog.append(anchor)
        after_anchor = comments[comments['reply_to_comment'] == int(anchor['comment_id'])].head(1)
        
    if len(pd.concat(dialog)) > 1:
        dialogs.append(pd.concat(dialog))
    comments = comments[~comments['comment_id'].isin(already_used_id)]
    anchor = comments[list(map(bool, comments['reply_to_comment']))].head(1)

TypeError: cannot convert the series to <class 'int'>

In [21]:
#only 2 users
already_used_id = []
dialogs = []
comments = source_comments.copy()
#loop across comments
#dialog = []
#comments = comments[~comments['comment_id'].isin(already_used_id)]
#anchor = comments[list(map(bool, comments['reply_to_comment']))].head(1)
#already_used_id.append(int(anchor['comment_id']))
#first = comments[comments['comment_id'] == int(anchor['reply_to_comment'])]
#dialog.append(first)
#dialog.append(anchor)
#after_anchor = comments[comments['reply_to_comment'] == int(anchor['comment_id'])].head(1)
#while(not after_anchor.dropna().empty):
#    anchor = after_anchor
#    already_used_id.append(int(anchor['comment_id']))
#    dialog.append(anchor)
#    after_anchor = comments[comments['reply_to_comment'] == int(anchor['comment_id'])].head(1)
    

comments = comments[~comments['comment_id'].isin(already_used_id)]
anchor = comments[list(map(bool, comments['reply_to_comment']))].head(1)
while(not anchor['reply_to_comment'].dropna().empty):
    dialog = []
    comments = comments[~comments['comment_id'].isin(already_used_id)]
    anchor = comments[list(map(bool, comments['reply_to_comment']))].head(1)
    already_used_id.append(int(anchor['comment_id']))
    first = comments[comments['comment_id'] == int(anchor['reply_to_comment'])].head(1)
    dialog.append(first)
    dialog.append(anchor)
    after_anchor = comments[comments['reply_to_comment'] == int(anchor['comment_id'])].head(1)
    filter_user = (comments['from_id'] == int(anchor['reply_to_user']))\
                    | (comments['from_id'] == int(anchor['from_id']))
    filter_post = comments['post_id'] == int(anchor['post_id'])
    while(not after_anchor.dropna().empty):
        anchor = after_anchor
        already_used_id.append(int(anchor['comment_id']))
        dialog.append(anchor)
        filter_reply = comments['reply_to_comment'] == int(anchor['comment_id'])
        after_anchor = comments[filter_reply & filter_user & filter_post].head(1)
        
    if len(pd.concat(dialog)) > 1:
        dialogs.append(pd.concat(dialog))
    comments = comments[~comments['comment_id'].isin(already_used_id)]
    anchor = comments[list(map(bool, comments['reply_to_comment']))].head(1)

In [22]:
def post_processing(dialogs):
    res=[]
    dialog=[]
    dialog.append(dialogs[0])
    for cur in dialogs[1:]:
        if set(dialog[-1]['from_id']) == set(cur['from_id'])\
            and set(dialog[-1]['post_id']) == set(cur['post_id']):
            dialog.append(cur)
        else:
            res.append(pd.concat(dialog))
            dialog=[cur]
    return res

In [23]:
dialogs = post_processing(dialogs)

In [24]:
def view_dialog(table):
    if len(table) > 1:
        res = []
        for _, i in table.iterrows():
            if i['text']:
                res.append(i['text'])
            else:
                return
        return res

In [25]:
with open('./view_only2users_and_post_proc.txt', 'w') as fw:
    for num, i in enumerate(dialogs):
        a = view_dialog(i)
        if a:
            for line in a:
                fw.write(line+'\n')
        fw.write(str(num)+"   "+'*'*10+'\n\n\n\n'+"*"*10+'\n')

In [14]:
dialogs[155]

,comment_id,date,from_id,num_answer,owner_id,parents_stack,post_id,reply_to_comment,reply_to_user,text
405,3824,1465803417,263098980,0,6154,[],3821,None,None,Что за тачка ?)
748,3825,1535128289,10210,0,10210,[],3823,3824,802213,"Тёмыч, на здоровье) попроси хорошо еще и место..."
749,3826,1535128610,802213,0,10210,[],3823,3825,10210,"Варвара, сказать не интересно. Ты пальцем пока..."
750,3827,1535129285,10210,0,10210,[],3823,3826,802213,"Тёмыч, приличные девочки пальцем не показывают..."
